In [1]:
from datasets import load_dataset

ds = load_dataset("climatebert/climate_detection")

c:\100 RAGs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\100 RAGs\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mukesh Bhai\.cache\huggingface\hub\datasets--climatebert--climate_detection. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [7]:
from langchain.schema import Document

# Confirm columns
print(ds["train"].column_names)  # ['text', 'label']

# Extract texts from the 'text' column
texts = ds["train"]["text"]

# Convert texts to Document objects
documents = [Document(page_content=text) for text in texts]


['text', 'label']


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000,
                                               chunk_overlap = 200)
splits = text_splitter.split_documents(documents)

In [10]:
print(splits[0])
print(splits[1])

page_content='− Scope 3: Optional scope that includes indirect emissions associated with the goods and services supply chain produced outside the organization. Included are emissions from the transport of products from our logistics centres to stores (downstream) performed by external logistics operators (air, land and sea transport) as well as the emissions associated with electricity consumption in franchise stores.'
page_content='The Group is not aware of any noise pollution that could negatively impact the environment, nor is it aware of any impact on biodiversity. With regards to land use, the Group is only a commercial user, and the Group is not aware of any local constraints with regards to water supply. The Group does not believe that it is at risk with regards to climate change in the near-or mid-term.'


In [11]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings 

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",google_api_key="XXXXX")  
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [12]:
print(vectorstore._collection.get())

{'ids': ['505145c1-758e-4984-8a45-0c5218ada760', '4bbcb3e9-ab14-47d7-a119-b101d878e5e7', '43aef411-13ea-43c4-aa27-b02996f9aab3', '1dda2ef1-0bad-4d62-8319-642640904737', '2fc3431b-770e-4e55-8c21-acf480d4d5eb', 'b98ff321-72a2-4b0b-a635-921e4f9a34e2', '88f40921-bbd6-431e-b57d-8114cd751211', 'a460e736-f7e1-4bf2-80d4-ac63c70df69e', '37c2beeb-02e8-43ec-b1fe-a52787aaa140', '4dc6ad94-32dd-4878-a6a5-614259f238f4', 'df4acbdc-0985-44c4-9d52-e0635caac057', 'c57c27bf-675b-4278-b92e-394a20542ce9', 'ae2122c4-e1d6-4928-87de-5a901ca8d468', '446b49f4-883d-475e-9ef4-e64ee4606577', 'f47a91a3-e896-40a8-bca9-6d82750d704f', 'a01fb2fe-89af-467c-979b-ce8002634a8c', '638037f6-dbfd-48fa-8fad-ab78a5f2b3db', '780c2025-acec-48a4-9622-9e39f3ae9ee5', '0f2bd9b9-3f93-4122-9b78-13d88f147204', '2d5571c2-d060-407a-80f1-0ad1ee53a735', '96347297-9322-4971-839e-704b20ca36c1', '128ec8b6-f260-41c5-a5c9-f73edfd63c3d', 'a38d9cc0-bdd3-41a7-934c-ad333975e40c', 'cbacdb05-e556-4c8a-af15-9d53cf6afab2', '17fbd1ff-4eba-46c6-985f-551084

In [13]:
retriever = vectorstore.as_retriever()

In [14]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\100 RAGs\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key="XXXXX"
)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [17]:
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

In [18]:
rag_chain = ({"context":retriever | format_docs, "question":RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [20]:
rag_chain.invoke("Carbon emissions are rising due to deforestation.")

'Yes, carbon emissions have tripled since 1960, contributing to climate change. Deforestation contributes to this issue, with forests shrinking by 13 million hectares per year. This unsustainable consumption of natural resources drives land and biodiversity loss.'